# 层和块

首先，我们回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 9.5607e-02, -2.2975e-01, -1.1883e-01, -3.4584e-02,  1.2305e-01,
          2.5676e-02, -3.3238e-02,  1.4586e-01,  3.0254e-02, -3.6541e-02],
        [ 2.8813e-02, -2.5555e-01, -1.7297e-01, -7.0474e-02,  1.7566e-01,
          2.0750e-01, -3.3867e-04,  2.9421e-01,  2.6093e-04,  5.2506e-02]],
       grad_fn=<AddmmBackward>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 1.3425e-01,  1.5510e-01,  9.5493e-02, -1.2406e-01,  1.4533e-01,
          1.7049e-01, -1.8365e-04, -5.5875e-02, -6.8056e-03, -2.1607e-01],
        [ 1.3548e-01,  2.4316e-01,  2.3426e-01, -1.2192e-01,  1.5611e-01,
          2.1362e-01, -6.9717e-02, -1.1583e-01, -1.1797e-01, -2.1189e-01]],
       grad_fn=<AddmmBackward>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2900, -0.0375,  0.0178,  0.0342,  0.0099,  0.0339, -0.0560,  0.0576,
         -0.0680,  0.0161],
        [-0.3584, -0.1464,  0.0839,  0.0921,  0.1037, -0.0658,  0.0064,  0.1065,
          0.0004,  0.1495]], grad_fn=<AddmmBackward>)

在正向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0162, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0566, grad_fn=<SumBackward0>)